# Прв парцијален испит по Основи на Вештачката Интелигенција

## Задача 1 - Преминување преку река

Две тројки, црна и бела, сочинети од крал, кралица и пиун треба да ја преминат реката.
* Кралевите не смеат да останат без своите поданици на една страна од реката.
* Кралиците сакаат да бидат сами кога ја преминуваат реката.
* Пиуните и кралиците знаат да веслаат, кралевите не веслаат.
* Чамецот може да издржи највеќе двајца.

Напишете алгоритам кој ќе го пронајде најмалиот број преминувања на реката, така што сите патници ќе преминат од левата на десната страна од реката.

In [6]:
from collections import deque
import itertools

In [15]:
def unpack_state(state):
    """
    Function that carefully unpacks the state into boat, left side and right side
    :param state: state to be unpack
    :return: boat, left side objects, right side objects
    """
    boat = state[0]
    left_bank = set([passengers[index] for index, side in enumerate(state[1:]) if side == 'left'])
    right_bank = set([passengers[index] for index, side in enumerate(state[1:]) if side == 'right'])
    return boat, left_bank, right_bank

def search_path(initial_state, goal_state):
    """
    Search function
    :param initial_state: initial state for search
    :param goal_state: desired state
    :return: search result as list of states
    """
    visited = {initial_state}
    states_queue = deque([[initial_state]])
    while states_queue:
        states_list = states_queue.popleft()
        state_to_expand = states_list[-1]
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if next_state == goal_state:
                    return states_list + [next_state]
                visited.add(next_state)
                states_queue.append(states_list + [next_state])
    return []

def visualise(path):
    """
    Function to visualise path returned from the search function
    :param path: path to be visualised
    :return: None
    """
    if not path:
        print('Search path did not find a solution')
        return
    for pair_of_states in zip(path, path[1:]):
        boat_old, left_old, right_old = unpack_state(pair_of_states[0])
        boat_new, left_new, right_new = unpack_state(pair_of_states[1])
        delimiter_space = ' ' * 50
        separated_print(left_old)
        print(delimiter_space, end='')
        separated_print(right_old)
        print()
        if boat_old == 'left':
            delimiter = ' ' * 5 + '>' * 15 + ' ' * 5
            separated_print(left_new)
            print(delimiter, end='')
            separated_print(left_old - left_new)
            print(delimiter, end='')
            separated_print(right_old)
            print()
        else:
            delimiter = ' ' * 5 + '<' * 15 + ' ' * 5
            separated_print(left_old)
            print(delimiter, end='')
            separated_print(right_old - right_new)
            print(delimiter, end='')
            separated_print(right_new)
            print()
        separated_print(left_new)
        print(delimiter_space, end='')
        separated_print(right_new)
        print()
        print()
        print()

def separated_print(iterable):
    """
    Desired print function.
    :param iterable: list to be printed
    :return: None
    """
    for element in iterable:
        print(element, end=' ')
    if not iterable:
        print('Empty', end='')

In [16]:
def obtain_boat_trips(available_passengers):
    boat_trips = [(passenger,) for passenger in available_passengers if passenger in sailors]
    for possible_boat_trip in itertools.combinations(available_passengers, 2):
#         Тука се итерира низ сите можни комбинации од две фигури кои можат да се качат на бродот.
#         Ако една од тие фигури е кралица, тогаш оваа комбинација ја исфрламе бидејќи кралиците сакаат сами да се возат на бродот.
        if any(['Queen' in passenger for passenger in possible_boat_trip]):
            continue
        sailors_on_boat = any([passenger in sailors for passenger in possible_boat_trip])
        if sailors_on_boat:
            boat_trips.append(possible_boat_trip)
    return boat_trips

def permited_river_bank(river_bank):
#     Тука испитуваме дали на дадената страна river_bank има крал кој ја нема до него неговата кралица или пак неговиот поданик.
    if 'Black King' in river_bank and 'Black Queen' not in river_bank and 'Black Pawn' not in river_bank:
        return False
    if 'White King' in river_bank and 'White Queen' not in river_bank and 'White Pawn' not in river_bank:
        return False
    return True

def belongs(passenger, left_side, right_side):
    if passenger in left_side:
        return 'left'
    elif passenger in right_side:
        return 'right'
    else:
        raise RuntimeError('Mayday, mayday, mayday. Passenger LOST')

def other_side(side):
    return 'left' if side == 'right' else 'right'

def expand_state(state):
    """
    Function for generation next states.

    Your should return a list of states as tuples. List new_states should look like this:
    [(state_parameter_1, state_parameter_2, ... state_parameter_n),
    (state_parameter_1, state_parameter_2, ... state_parameter_n)),
    .
    .
    .
    (state_parameter_1, state_parameter_2, ... state_parameter_n))]

    :param state: state to be expanded
    :return: list of new states as tuples
    """
    # За полесни пресметки јас вака ги претставив објектите. Слободно променете ако имате подобра идеја.
    boat, left_bank, right_bank = unpack_state(state)

    new_states = []

    # Вашиот код тука
    
    if boat == 'left':
        boat_trips = obtain_boat_trips(left_bank)
    else:
        boat_trips = obtain_boat_trips(right_bank)

    new_states = []
    for boat_trip in boat_trips:
        if boat == 'left':
            new_left_bank = left_bank - set(boat_trip)
            new_right_bank = right_bank | set(boat_trip)
        else:
            new_left_bank = left_bank | set(boat_trip)
            new_right_bank = right_bank - set(boat_trip)

        if permited_river_bank(new_left_bank) and permited_river_bank(new_right_bank):
            black_king = belongs('Black King', new_left_bank, new_right_bank)
            black_queen = belongs('Black Queen', new_left_bank, new_right_bank)
            black_pawn = belongs('Black Pawn', new_left_bank, new_right_bank)
            white_king = belongs('White King', new_left_bank, new_right_bank)
            white_queen = belongs('White Queen', new_left_bank, new_right_bank)
            white_pawn = belongs('White Pawn', new_left_bank, new_right_bank)
            new_states.append((other_side(boat), black_king, black_queen, black_pawn, white_king, white_queen, white_pawn))

    return new_states

passengers = ['Black King', 'Black Queen', 'Black Pawn', 'White King', 'White Queen', 'White Pawn']
# Преку листата sailors означуваме дека кралевите не знаат да веслаат.
sailors = ['Black Queen', 'Black Pawn', 'White Queen', 'White Pawn']
initial_state = ('left', 'left', 'left', 'left', 'left', 'left', 'left')
goal_state = ('right', 'right', 'right', 'right', 'right', 'right', 'right')
path = search_path(initial_state, goal_state)
visualise(path)

White King Black Queen White Queen Black King White Pawn Black Pawn                                                   Empty
White Queen Black King Black Queen White King      >>>>>>>>>>>>>>>     White Pawn Black Pawn      >>>>>>>>>>>>>>>     Empty
White Queen Black King Black Queen White King                                                   White Pawn Black Pawn 


White Queen Black King Black Queen White King                                                   White Pawn Black Pawn 
White Queen Black King Black Queen White King      <<<<<<<<<<<<<<<     White Pawn      <<<<<<<<<<<<<<<     Black Pawn 
White King Black Queen White Queen Black King White Pawn                                                   Black Pawn 


White King Black Queen White Queen Black King White Pawn                                                   Black Pawn 
White Pawn Black King Black Queen White King      >>>>>>>>>>>>>>>     White Queen      >>>>>>>>>>>>>>>     Black Pawn 
White Pawn Black King Black Queen 

## Забелешка

Во оригиналната игра има правило дека еден крал не може да се најде на чамецот заедно со пиун од другата боја. Ама ова барање ненамерно е испуштено од горенаведените барања. Така и го оставив моето решение. Да не ве чуди ако во решението видете на чамецот крал и пиун во различна боја.